In [ ]:
import cv2 as cv
import matplotlib 
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from collections import namedtuple
from operator import itemgetter
from pprint import pformat
from numpy import inf

In [ ]:
def getScale(v,kbits):
    #f = round(255/pow(2,round(kbits/3)));
    #r = round(v[0] / f);
    #g = round(v[1] / f);
    #b = round(v[2] / (f));
    colorBits = round(kbits/3);
    mask = pow(2,(colorBits))-1;
    mask = mask << (8-colorBits);
    remainingBIts = (kbits - 2*colorBits)
    bmask = pow(2,remainingBIts)-1; 
    bmask = bmask << (8-remainingBIts)
    r = v[0] & mask;
    g = v[1] & mask;
    b = v[2] & bmask;
    #print(mask,'-',bmask)
    #print(f) 
    #print(v[0],v[1],v[2],'->',r,g,b)
    return [r,g,b];

def scaleDownImage(img,kbits):
    h = img.shape[0]
    l = img.shape[1]
    imgNew = 0 * img;
    for i in range(h):
        for j in range(l):
            imgNew[i,j] = getScale(img[i,j],kbits);
    #print(imgNew.shape)        
    return imgNew       

def histogram(img):
    hist2 = {}
    h = img.shape[0]
    l = img.shape[1]
    for i in range(h):
        for j in range(l):
            s = str(img[i,j,0])+'_'+str(img[i,j,1])+'_'+str(img[i,j,2])
            if(s in hist2.keys()):
                hist2[s] = hist2[s] + 1;
            else:
                 hist2[s] = 1
    #print(hist2)                
    return hist2

def findkpopular(h,kColours):
        i = kColours
        l = []
        while(i>0):
            if(len(h.keys())==0):
                return l
            maxk = max(h, key=h.get) 
            #print(Keymax)
            maxList = list(map(int, maxk.split('_')))
            l.append(maxList)
            #print("max key",maxk)
            del h[maxk] 
            i=i-1;
        return l
    
def getColour(newColours,v):
    #print(v)
    newV = []
    newV = min(newColours, key=lambda x:abs((((x[0] - v[0])) + ((x[1]-v[1])) + ((x[2]-v[2])))))
    return newV


def redraw(img,tree):
    h = img.shape[0]
    l = img.shape[1]
    imgNew = 0 * img;
    for i in range(h):
        for j in range(l):
            #imgNew[i,j] = getColour(newColours,img[i,j]);
            minDist,point = findNearest(tree,img[i,j],inf,[],0)
            imgNew[i,j] = point
    #print(imgNew.shape)        
    return imgNew 




def limitV(vec):
    for i in range(len(vec)):
        if(vec[i]>255):
            vec[i] = 255
        if(vec[i]<0):
            vec[i] = 0
    return vec    
        

def redrawWithFloydAndStienBerg(imgOriginal,tree):
    h = imgOriginal.shape[0]
    l = imgOriginal.shape[1]
    k = len(imgOriginal[0,0])
    img = 1 * imgOriginal
    imgNew = 0 * img;
    for i in range(h):
        for j in range(l):
            minDist,point = findNearest(tree,img[i,j],inf,[],0)
            imgNew[i,j] = point
            error = []
            x = img[i,j]
            v = imgNew[i,j]
            for q in range(k):
                error.append((int(x[q]) - int(v[q]))) 
            #if(error != [0,0,0]):
            #print(x,"-",error,"-",v)    
            if((j+1) < l):
                #print(img[i,j+1])
                img[i,j+1] = limitV(np.add(np.array(img[i,j+1],dtype="int") , np.array([round(e*(3.0/8)) for e in error],dtype="int"),dtype="int").tolist())
                #print(img[i,j+1])
                #return imgNew
            if(i+1<h):     
                img[i+1,j] = limitV(np.add(np.array(img[i+1,j],dtype="int") , np.array([round(e*(3.0/8)) for e in error],dtype="int"),dtype="int").tolist())
            if(i+1<h and j+1<l):
                img[i+1,j+1] = limitV(np.add(np.array(img[i+1,j+1],dtype="int") , np.array([round(e*(1.0/4))  for e in error],dtype="int"),dtype="int").tolist())
    #print(imgNew.shape)        
    return imgNew



class Node(namedtuple("Node", "location left_child right_child")):
    def __repr__(self):
        return pformat(list(self))

def kdtree(point_list, depth: int = 0):
    if not point_list:
        return None

    k = len(point_list[0])  # assumes all points have the same dimension
    # Select axis based on depth so that axis cycles through all valid values
    axis = depth % k

    # Sort point list by axis and choose median as pivot element
    point_list.sort(key=itemgetter(axis))
    median = len(point_list) // 2

    # Create node and construct subtrees
    return Node(
        location=point_list[median],
        left_child=kdtree(point_list[:median], depth + 1),
        right_child=kdtree(point_list[median + 1 :], depth + 1),
    )

def findNearest(tree,point,minDist,closestPoint,depth):
        if(tree is None):
            return minDist,closestPoint
        else:
            x = tree.location
            v = point
            k = len(point)
            getdistance = 0
            for i in range(k):
                getdistance = getdistance + ((x[i]-v[i])**2.0)    
            getdistance = abs((getdistance)**0.5)
            #getdistance = abs((((x[0] - v[0]))**2.0 + ((x[1]-v[1]))**2.0 + ((x[2]-v[2]))**2.0 )**0.5)
            if getdistance < minDist:
                minDist = getdistance
                closestPoint = tree.location
            axis = depth % k
            if((tree.location)[axis] > point[axis]):
                return findNearest(tree.left_child,point,minDist,closestPoint,depth+1)
            elif((tree.location)[axis] < point[axis]): 
                return findNearest(tree.right_child,point,minDist,closestPoint,depth+1)
            else:
                if(tree.left_child):
                     minDist,closestPoint=findNearest(tree.left_child,point,minDist,closestPoint,depth+1)
                        
                if(tree.right_child):
                    minDistr,closestPointr=findNearest(tree.right_child,point,minDist,closestPoint,depth+1) 
                    if(minDist < minDistr):
                        minDist = minDistr
                        closestPoint = closestPointr
                    
                return minDist,closestPoint

            
            

In [ ]:
def runall(filename,k):
    #imCv1 = cv.imread('pamela.jpg') 
    imCv1 = cv.imread(filename) 
    img1 = cv.cvtColor(imCv1, cv.COLOR_BGR2RGB)
    scaledImg1 = scaleDownImage(img1,15);
    histo1 = histogram(img1)
    histoScaled1 = histogram(scaledImg1)
    print("# of Colours in Image :",len(histo1),", Number of Coulours in Scaled Iamge :",len(histoScaled1))
    colorMap1 = findkpopular(histo1,pow(2,k))
    colorMapScaled1 = findkpopular(histoScaled1,pow(2,k))
    print("# of colours ColourMap:",len(colorMap1),", # of colours ColourMap(scaled):",len(colorMapScaled1))
    tree1 = kdtree(colorMap1)
    treeScaled1 = kdtree(colorMapScaled1)
    myNewIamge1 = redraw(img1,tree1);
    myNewIamgeScaled1 = redraw(scaledImg1,treeScaled1);
    ditheredImage = redrawWithFloydAndStienBerg(img1,tree1)
    ditheredImageScaled = redrawWithFloydAndStienBerg(scaledImg1,treeScaled1)
    f, axArr = plt.subplots(nrows=3,ncols=2,figsize=(50,50))
    plt.sca(axArr[0][0]); 
    plt.imshow(img1); 
    plt.title('Original Image',fontsize=50)
    plt.sca(axArr[0][1]); 
    plt.imshow(myNewIamge1); 
    plt.title('k('+str(2**k)+')-Popular Image',fontsize=50)
    plt.sca(axArr[1][0]); 
    plt.imshow(scaledImg1); 
    plt.title('Scaled Image - 15 bits',fontsize=50)
    plt.sca(axArr[1][1]); 
    plt.imshow(myNewIamgeScaled1); 
    plt.title('Scaled k('+str(2**k)+')-Popular Image',fontsize=50)
    plt.sca(axArr[2][0]); 
    plt.imshow(ditheredImage); 
    plt.title('k('+str(2**k)+')-Popular Floyd Stienberg Dither Image',fontsize=50)
    plt.sca(axArr[2][1]); 
    plt.imshow(ditheredImageScaled); 
    plt.title('Scaled k('+str(2**k)+')-Popular Floyd Stienberg Dither Image',fontsize=50)
    plt.show()
    
    return img1,scaledImg1,myNewIamge1,myNewIamgeScaled1,tree1,treeScaled1

In [ ]:
originalIamge,originalIamgeScaled,finalImage,fianlScaledIamge,colorTree,colorTreeScaled = runall('pamela.jpg',3)

In [ ]:
originalIamge,originalIamgeScaled,finalImage,fianlScaledIamge,colorTree,colorTreeScaled = runall('pamela.jpg',4)

In [ ]:
originalIamge,originalIamgeScaled,finalImage,fianlScaledIamge,colorTree,colorTreeScaled = runall('pamela.jpg',6)

In [ ]:
originalIamge,originalIamgeScaled,finalImage,fianlScaledIamge,colorTree,colorTreeScaled = runall('pamela.jpg',8)

In [ ]:
originalIamge,originalIamgeScaled,finalImage,fianlScaledIamge,colorTree,colorTreeScaled = runall('pamela.jpg',10)

In [ ]:
originalIamge,originalIamgeScaled,finalImage,fianlScaledIamge,colorTree,colorTreeScaled = runall('pamela.jpg',12)